In [1]:
import numpy as np
from PIL import Image
from glob import glob
import re
import json
from tqdm import tqdm_notebook as tqdm
from collections import *

In [2]:
images = ['./data/0.png', './data/1.png', './data/2.png', './data/3.png', './data/4.png', './data/5.png', './data/6.png', './data/7.png', './data/8.png', './data/9.png']

In [3]:
rows = [490, 667, 850, 1034]
cols = [118, 330, 540]

In [4]:
pixel_locs = [
     [rows[3], cols[1]], 
     [rows[0], cols[0]], 
     [rows[0], cols[1]], 
     [rows[0], cols[2]], 
     [rows[1], cols[0]],
     [rows[1], cols[1]],
     [rows[1], cols[2]],
     [rows[2], cols[0]],
     [rows[2], cols[1]], 
     [rows[2], cols[2]]
    ]

In [5]:
#thresholds = [x/3 * 2 for x in [180, 150, 200, 170, 160, 190, 170, 200, 200, 170]]

In [6]:
def classify_image(path):
    hits = []
    opened_img = np.asarray(Image.open(path))
    for i in range(10):
        avg = np.average(opened_img[int(pixel_locs[i][0]), int(pixel_locs[i][1])]) 
        #print(i, avg)
        if avg > 50:
            hits.append(i)
    if len(hits) >= 1:
        return hits[0]
    elif len(hits) == 0:
        return 10
    else:
        raise Exception(f'Multiple classes for image {path}')
    return hits

In [7]:
#classify_image('./data/vidya/image-00120.jpg')

In [8]:
#[classify_image(x) for x in images]

In [9]:
def classify_many(person_path, screen_path, frames_person_ahead_of_screen, out_path):
    #framerates are 23.12 (person) and 59.87 (screen)
    person_files = sorted(glob(person_path), key=lambda x: int(re.findall('\d+', x)[0]))
    screen_files = sorted(glob(screen_path), key=lambda x: int(re.findall('\d+', x)[0]))
    truth_at_person_frame = defaultdict(lambda: 10)
    
    counter = -1 * frames_person_ahead_of_screen
    for f in tqdm(screen_files):
        result = classify_image(f)
        truth_at_person_frame[counter] = result
        counter += 1
        #print(result)

    with open(out_path, 'w') as out_f:
        out_f.write("id,val\n")
        
        for p in range(len(person_files)):
            out_f.write(f'{p}, class_{truth_at_person_frame[p]}\n')

In [10]:
classify_many('./data/vidya/person/*.jpg', './data/vidya/screen/*.jpg', 312-21, './data/y3.csv')